# Moody (2001) — RL para trading con objetivo tipo Sharpe
Baseline educativo sin redes. Política lineal con tanh, entrenamiento en línea con proxy diferenciable del Sharpe. Acciones continuas en [-1,1] como fracción de capital expuesto. Incluye costos de transacción.

**Contenido**
1) Datos sintéticos
2) Señal paramétrica s_t = tanh(w^T x_t)
3) PnL con costos y EMA de media/varianza
4) Paso de gradiente sobre Sharpe proxy
5) Backtest y métricas


## Imports

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
rng = np.random.default_rng(123)


## Datos sintéticos

In [ ]:

def synthetic_returns(n=5000, ar=0.05, sigma=0.012, jumps=0.0005, jscale=0.05, seed=123):
    rng = np.random.default_rng(seed)
    r = np.zeros(n)
    for t in range(1,n):
        r[t] = ar*r[t-1] + rng.normal(0,sigma)
        if rng.random() < jumps:
            r[t] += rng.normal(0, jscale)
    return r

r = synthetic_returns()
plt.figure(); plt.plot(r[:600]); plt.title('Rendimientos sintéticos'); plt.show()


## Señal, PnL y entrenamiento tipo Moody 2001

In [ ]:

def features(r, k=12):
    X = []
    for t in range(k, len(r)):
        X.append(r[t-k:t][::-1])
    return np.array(X), r[k:]

def pnl_from_signal(signal, ret, cost=0.0002):
    ds = np.diff(signal, prepend=0.0)
    return signal*ret - cost*np.abs(ds)

def train_moody2001(r, k=12, lr=0.03, ema=0.02, beta=1e-3, cost=0.0002, epochs=3):
    X, y = features(r, k=k)
    w = np.zeros(k)
    m, v = 0.0, 1e-4
    SR_track, w_track = [], []
    for ep in range(epochs):
        for t in range(len(y)):
            x = X[t]
            s = np.tanh(x @ w)
            s_prev = np.tanh((X[t-1] @ w) if t>0 else 0.0)
            ds = s - s_prev
            pnl = s*y[t] - cost*np.abs(ds)
            # EMA momentos
            m = (1-ema)*m + ema*pnl
            v = (1-ema)*v + ema*(pnl - m)**2
            sr = m/np.sqrt(v + beta)
            SR_track.append(sr)

            # Gradiente (proxy): d pnl/d w
            ds_sign = np.tanh(1000*ds)
            dpnl_dw = (1 - s**2) * x * y[t] - cost * ds_sign * (1 - s**2) * x
            # Cadena para SR
            d_sr_dm = 1/np.sqrt(v + beta)
            d_sr_dv = -0.5*m*(v+beta)**(-1.5)
            d_sr_dpnl = d_sr_dm*ema + d_sr_dv*(2*ema*(pnl - m))
            grad = d_sr_dpnl * dpnl_dw
            w += lr*grad
            if t % 500 == 0: w_track.append(w.copy())
    return w, np.array(SR_track), X, y

w, SR_tr, X, y = train_moody2001(r, epochs=4)
signal = np.tanh(X @ w)
pnl = pnl_from_signal(signal, y)
print("Sharpe ex-post:", pnl.mean()/(pnl.std()+1e-9))

plt.figure(); plt.plot(np.cumsum(pnl)); plt.title('Equity curve'); plt.show()
